<a href="https://colab.research.google.com/github/Sumit-kumar99/LangChain-RAG/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 👣 STEP 1: Install dependencies
!pip install -q langchain faiss-cpu sentence-transformers

In [ ]:
# 👣 STEP 2: Upload a text file
from google.colab import files
uploaded = files.upload()

Saving LangC.txt to LangC (3).txt


In [ ]:
# 👣 STEP 3: Read the uploaded file
filename = list(uploaded.keys())[0]
with open(filename, 'r') as f:
    raw_text = f.read()

In [ ]:
# Install the missing package
!pip install -U langchain-huggingface

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline

In [ ]:
# Step 4: Split
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents = [Document(page_content=chunk) for chunk in splitter.split_text(raw_text)]

In [ ]:
# Step 5: Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)

In [ ]:
# Step 6: Use stronger FLAN model
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [ ]:
# Step 7: Build QA
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

In [ ]:
# Step 8: Ask question
question = "What is the leave policy in this document?"
result = qa_chain.invoke({"query": question})
print("🧠 Answer:", result)

🧠 Answer: {'query': 'What is the leave policy in this document?', 'result': 'Leave Policy Employees are entitled to 24 paid leaves annually, including 12 casual leaves, 6 sick leaves, and 6 earned leaves. Leaves must be requested at least 2 days in advance through the HR portal. Section 2: Leave Policy Employees are entitled to 24 paid leaves annually, including 12 casual leaves, 6 sick leaves, and 6 earned leaves. Leaves must be requested at least 2 days in advance through the HR portal. Section 3: Code of Conduct All employees must behave professionally. Harassment of any kind will not be tolerated and may result in termination.'}


In [ ]:
# Step 9: Ask question
question = "How many days of leave do employees get?"
result = qa_chain.invoke({"query": question})
print("🧠 Answer:", result)

🧠 Answer: {'query': 'How many days of leave do employees get?', 'result': '24'}


In [ ]:
# Step 10: Ask question
question = "List the types of leave mentioned."
result = qa_chain.invoke({"query": question})
print("🧠 Answer:", result)

🧠 Answer: {'query': 'List the types of leave mentioned.', 'result': 'Casual leaves, 6 sick leaves, and 6 earned leaves'}


In [ ]:
# Step 11: Optional: Print what context was retrieved
print("\n📚 Retrieved Context:")
for doc in vectorstore.similarity_search(question, k=3):
    print("-", doc.page_content[:200], "...\n")


📚 Retrieved Context:
- Section 2: Leave Policy
Employees are entitled to 24 paid leaves annually, including 12 casual leaves, 6 sick leaves, and 6 earned leaves. Leaves must be requested at least 2 days in advance through t ...

- Section 5: Termination Clause
Either party may terminate the employment by providing a 30-day notice. In cases of policy violations or misconduct, the company reserves the right to terminate employmen ...

- Section 3: Code of Conduct
All employees must behave professionally. Harassment of any kind will not be tolerated and may result in termination. Confidential company data must not be shared outside wi ...

